# San Francisco Working Families Credit

Analysis of [eligibility criteria](https://www.sfhsa.org/services/jobs-money/working-families-credit-wfc-program/eligibility-wfc) for the San Francisco Working Families Credit:

> * Have never received the WFC before
* Have earned less than $53,930 in 2017
* Have filed your tax return by April 17 deadline
* Claim the Earned Income Tax Credit (EITC) on your federal income tax return (see table below for more information)
* Claim at least one qualifying dependent child on your federal income tax return
* Live in San Francisco

The income limit of \$53,930 is redundant with the EITC claiming, as this is the maximum AGI to claim the EITC, [per IRS guidelines](https://www.irs.gov/credits-deductions/individuals/earned-income-tax-credit/eitc-income-limits-maximum-credit-amounts).

This notebook uses national CPS data to estimate the share of all EITC filers that have a qualifying dependent child.

While these criteria concern 2017 thresholds, this analysis uses 2018 data and tax logic, as any reforms would be for the 2018 tax year (early 2019).

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-03-08*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# On Python 3.6 use "import urllib.request as url_lib".
import urllib as url_lib

In [2]:
tc.__version__

'0.16.1'

### Configuration

In [3]:
sns.set_style('white')
DPI = 75
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [6]:
# Show one decimal in tables.
pd.set_option('precision', 2)

### Constants

In [5]:
WFC_INC_C = 53930

## Data

Generate data using Tax-Calculator.

In [7]:
recs = tc.Records.cps_constructor()

In [8]:
calc = tc.Calculator(records=recs, policy=tc.Policy())
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [14]:
df = calc.dataframe([
    's006', 'aftertax_income', 'XTOT',
    'EIC',  # Children eligible for EITC.
    'eitc',  # EITC amount.
    'c00100',  # AGI, used for EITC threshold.
    # Include non-cash benefits to subtract for cash income.
    # Vet and other benefits have some cash components, but unclear how much.
    'snap_ben', 'mcaid_ben', 'mcare_ben', 'vet_ben', 'other_ben'])

Flags for WFC.

In [20]:
df['wfc_agi_eligible'] = df['c00100'] < WFC_INC_C
df['has_EIC'] = df['EIC'] > 0

Summed variables.

In [17]:
df['XTOT_m'] = df['XTOT'] * df['s006'] / 1e6
df['EIC_m'] = df['EIC'] * df['s006'] / 1e6
df['eitc_m'] = df['eitc'] * df['s006'] / 1e6
df['aftertax_income_m'] = df['aftertax_income'] * df['s006'] / 1e6

Limit to EITC recipients and compute shares of tax units and people.

In [31]:
eitcr = df[df['eitc'] > 0].copy()  # Avoids warning.
eitcr['s006_share'] = 100 * eitcr['s006'] / eitcr['s006'].sum()
eitcr['XTOT_share'] = 100 * eitcr['XTOT_m'] / eitcr['XTOT_m'].sum()

## Analysis

In [32]:
eitcr.groupby(['wfc_agi_eligible', 'has_EIC'])['s006_share', 
                                               'XTOT_share'].sum()

s006_share  XTOT_share
wfc_agi_eligible has_EIC                        
False            True           0.14        0.30
True             False         24.24       14.45
                 True          75.62       85.25

In [ ]:
eitcr['s006'].sum()

In [10]:
df.head()

,s006,aftertax_income,EIC,eitc,c00100,snap_ben,mcaid_ben,mcare_ben,vet_ben,other_ben
0,239.46,42287.55,0.0,0.00,48553.93,0.0,0.0,0.0,0.0,0.00
1,202.59,20461.07,1.0,3468.00,16884.75,0.0,0.0,0.0,0.0,0.00
2,255.35,96222.03,0.0,0.00,0.00,915.0,46676.0,1914.0,0.0,30202.76
3,178.36,26597.38,1.0,2746.84,23218.36,0.0,634.0,0.0,0.0,372.92
4,270.80,41631.34,0.0,0.00,1048.81,0.0,0.0,5828.0,0.0,0.00
